In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
selected_features = ['dst_host_same_src_port_rate',
 'is_guest_login',
 'num_failed_logins',
 'dst_host_diff_srv_rate',
 'rerror_rate',
 'logged_in',
 'serror_rate',
 'same_srv_rate',
 'protocol_type_0',
'protocol_type_1', 
 'dst_host_srv_diff_host_rate']

In [3]:
df = pd.read_csv("../../../transformed_datasets/transformed_NSL_KDD_2.csv")

In [4]:
feature_columns = []
for feature in df.columns:
    if df[feature].dtype == 'int64' or df[feature].dtype == 'float64':
        feature_columns.append(feature)

In [5]:
features_to_be_normalised = []
for feature in feature_columns:
    if not df[feature].min() >= 0.0 and df[feature].max() <= 1.0:
        features_to_be_normalised.append(feature)
df['label'].nunique()

38

In [6]:
# scaler = StandardScaler()
# df[features_to_be_normalised] = scaler.fit_transform(df[features_to_be_normalised])
# df.describe()

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [7]:
df = df.astype({col: 'float32' for col in df.select_dtypes(include=['int', 'float64']).columns})

In [8]:
for f in selected_features:
    print(df[f].dtype)

float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32


In [9]:
features_tensor = torch.tensor(df[selected_features].values)

In [10]:
target_tensor = torch.tensor(df['label'].values, dtype=torch.long)

In [11]:
learning_rate = 1e-3
batch_size = 64
epochs = 1000

In [12]:
dataset = TensorDataset(features_tensor, target_tensor)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle=True)

In [13]:
device = torch.device("mps")

In [28]:
class MultiLayerPerceptron(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 300),
            nn.ReLU(),
            nn.Linear(300, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 38),
        )
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
        

In [29]:
model = MultiLayerPerceptron().to(device)

In [30]:
loss_fn = nn.CrossEntropyLoss()

In [31]:
def train_model(dataloader, loss_fn, model, optimizer):
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Move data to the device
        prediction = model(X)
        loss = loss_fn(prediction, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    

In [32]:
def test_model(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  # Move data to the device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [33]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [34]:
for i in range(epochs):
    print(i)
    train_model(dataloader, loss_fn, model, optimizer)
    test_model(dataloader, model, loss_fn)

0
Test Error: 
 Accuracy: 43.1%, Avg loss: 3.457333 

1
Test Error: 
 Accuracy: 43.1%, Avg loss: 3.170875 

2
Test Error: 
 Accuracy: 43.1%, Avg loss: 2.571609 

3
Test Error: 
 Accuracy: 43.1%, Avg loss: 2.238367 

4
Test Error: 
 Accuracy: 43.1%, Avg loss: 2.136536 

5
Test Error: 
 Accuracy: 43.1%, Avg loss: 2.073757 

6
Test Error: 
 Accuracy: 43.1%, Avg loss: 2.018522 

7
Test Error: 
 Accuracy: 43.1%, Avg loss: 1.967332 

8
Test Error: 
 Accuracy: 56.4%, Avg loss: 1.914946 

9
Test Error: 
 Accuracy: 61.2%, Avg loss: 1.858919 

10
Test Error: 
 Accuracy: 63.3%, Avg loss: 1.798713 

11
Test Error: 
 Accuracy: 63.2%, Avg loss: 1.737422 

12
Test Error: 
 Accuracy: 63.1%, Avg loss: 1.677593 

13
Test Error: 
 Accuracy: 63.1%, Avg loss: 1.624322 

14
Test Error: 
 Accuracy: 63.1%, Avg loss: 1.578213 

15
Test Error: 
 Accuracy: 63.3%, Avg loss: 1.540275 

16
Test Error: 
 Accuracy: 63.3%, Avg loss: 1.505570 

17
Test Error: 
 Accuracy: 63.3%, Avg loss: 1.476274 

18
Test Error: 
 Acc

Test Error: 
 Accuracy: 80.6%, Avg loss: 0.683344 

150
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.680830 

151
Test Error: 
 Accuracy: 80.6%, Avg loss: 0.679133 

152
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.677449 

153
Test Error: 
 Accuracy: 80.6%, Avg loss: 0.676584 

154
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.675588 

155
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.674068 

156
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.673213 

157
Test Error: 
 Accuracy: 80.6%, Avg loss: 0.671290 

158
Test Error: 
 Accuracy: 80.6%, Avg loss: 0.670050 

159
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.667968 

160
Test Error: 
 Accuracy: 81.4%, Avg loss: 0.666363 

161
Test Error: 
 Accuracy: 80.8%, Avg loss: 0.666086 

162
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.664439 

163
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.663690 

164
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.662367 

165
Test Error: 
 Accuracy: 81.4%, Avg loss: 0.662124 

166
Test Error: 
 Accuracy: 80.8%, Avg loss: 0.65946

Test Error: 
 Accuracy: 83.0%, Avg loss: 0.575786 

297
Test Error: 
 Accuracy: 82.9%, Avg loss: 0.576041 

298
Test Error: 
 Accuracy: 82.9%, Avg loss: 0.574203 

299
Test Error: 
 Accuracy: 82.8%, Avg loss: 0.574679 

300
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.572663 

301
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.574875 

302
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.572459 

303
Test Error: 
 Accuracy: 82.9%, Avg loss: 0.573308 

304
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.572205 

305
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.571336 

306
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.573539 

307
Test Error: 
 Accuracy: 82.9%, Avg loss: 0.571511 

308
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.570861 

309
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.570591 

310
Test Error: 
 Accuracy: 82.9%, Avg loss: 0.569932 

311
Test Error: 
 Accuracy: 83.2%, Avg loss: 0.571826 

312
Test Error: 
 Accuracy: 83.2%, Avg loss: 0.569225 

313
Test Error: 
 Accuracy: 82.9%, Avg loss: 0.56921

Test Error: 
 Accuracy: 83.6%, Avg loss: 0.533596 

444
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.533248 

445
Test Error: 
 Accuracy: 83.8%, Avg loss: 0.532706 

446
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.532105 

447
Test Error: 
 Accuracy: 84.2%, Avg loss: 0.533509 

448
Test Error: 
 Accuracy: 83.8%, Avg loss: 0.533973 

449
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.532721 

450
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.533767 

451
Test Error: 
 Accuracy: 83.6%, Avg loss: 0.532480 

452
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.531882 

453
Test Error: 
 Accuracy: 83.8%, Avg loss: 0.530878 

454
Test Error: 
 Accuracy: 83.8%, Avg loss: 0.532465 

455
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.530423 

456
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.530208 

457
Test Error: 
 Accuracy: 83.6%, Avg loss: 0.532651 

458
Test Error: 
 Accuracy: 84.1%, Avg loss: 0.531584 

459
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.531650 

460
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.53147

Test Error: 
 Accuracy: 84.6%, Avg loss: 0.508574 

591
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.505834 

592
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.506400 

593
Test Error: 
 Accuracy: 84.8%, Avg loss: 0.504822 

594
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.520028 

595
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.506511 

596
Test Error: 
 Accuracy: 84.8%, Avg loss: 0.505020 

597
Test Error: 
 Accuracy: 84.8%, Avg loss: 0.503552 

598
Test Error: 
 Accuracy: 84.9%, Avg loss: 0.503356 

599
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.506516 

600
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.503855 

601
Test Error: 
 Accuracy: 84.6%, Avg loss: 0.504375 

602
Test Error: 
 Accuracy: 84.8%, Avg loss: 0.503426 

603
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.514114 

604
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.502632 

605
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.502558 

606
Test Error: 
 Accuracy: 84.9%, Avg loss: 0.502582 

607
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.50397

Test Error: 
 Accuracy: 85.4%, Avg loss: 0.479084 

738
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.478934 

739
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.480331 

740
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.478105 

741
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.479541 

742
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.477440 

743
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.477203 

744
Test Error: 
 Accuracy: 85.3%, Avg loss: 0.476721 

745
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.476934 

746
Test Error: 
 Accuracy: 85.4%, Avg loss: 0.477018 

747
Test Error: 
 Accuracy: 85.5%, Avg loss: 0.478319 

748
Test Error: 
 Accuracy: 85.4%, Avg loss: 0.477119 

749
Test Error: 
 Accuracy: 85.4%, Avg loss: 0.478291 

750
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.476063 

751
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.475977 

752
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.475203 

753
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.476640 

754
Test Error: 
 Accuracy: 85.3%, Avg loss: 0.47514

Test Error: 
 Accuracy: 85.6%, Avg loss: 0.451400 

885
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.451811 

886
Test Error: 
 Accuracy: 85.5%, Avg loss: 0.452934 

887
Test Error: 
 Accuracy: 85.4%, Avg loss: 0.451611 

888
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.452658 

889
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.452737 

890
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.450830 

891
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.451169 

892
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.451379 

893
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.450209 

894
Test Error: 
 Accuracy: 85.9%, Avg loss: 0.451819 

895
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.451842 

896
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.449497 

897
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.450117 

898
Test Error: 
 Accuracy: 86.0%, Avg loss: 0.451962 

899
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.454798 

900
Test Error: 
 Accuracy: 85.9%, Avg loss: 0.450325 

901
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.44999

In [35]:
for i in range(epochs):
    print(i)
    train_model(dataloader, loss_fn, model, optimizer)
    test_model(dataloader, model, loss_fn)

0
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.435392 

1
Test Error: 
 Accuracy: 86.8%, Avg loss: 0.433949 

2
Test Error: 
 Accuracy: 86.6%, Avg loss: 0.437641 

3
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.437247 

4
Test Error: 
 Accuracy: 86.6%, Avg loss: 0.434624 

5
Test Error: 
 Accuracy: 86.6%, Avg loss: 0.433201 

6
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.432982 

7
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.434032 

8
Test Error: 
 Accuracy: 86.6%, Avg loss: 0.432854 

9
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.434257 

10
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.432771 

11
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.433897 

12
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.433078 

13
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.431649 

14
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.434062 

15
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.432548 

16
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.431414 

17
Test Error: 
 Accuracy: 86.6%, Avg loss: 0.434085 

18
Test Error: 
 Acc

Test Error: 
 Accuracy: 87.0%, Avg loss: 0.415082 

150
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.416444 

151
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.418033 

152
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.418809 

153
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.415522 

154
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.427771 

155
Test Error: 
 Accuracy: 86.8%, Avg loss: 0.419284 

156
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.413843 

157
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.418390 

158
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.417025 

159
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.415503 

160
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.414418 

161
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.416121 

162
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.421322 

163
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.414812 

164
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.413265 

165
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.416070 

166
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.41346

Test Error: 
 Accuracy: 87.1%, Avg loss: 0.399113 

297
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.402194 

298
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.399039 

299
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.402010 

300
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.400944 

301
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.403149 

302
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.398098 

303
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.401787 

304
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.399194 

305
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.404649 

306
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.409649 

307
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.401055 

308
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.397231 

309
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.410844 

310
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.397644 

311
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.397312 

312
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.397755 

313
Test Error: 
 Accuracy: 86.8%, Avg loss: 0.39905

Test Error: 
 Accuracy: 87.2%, Avg loss: 0.383807 

444
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.396991 

445
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.383253 

446
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.387432 

447
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.386805 

448
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.399447 

449
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.384692 

450
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.382980 

451
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.384170 

452
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.396270 

453
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.383155 

454
Test Error: 
 Accuracy: 86.6%, Avg loss: 0.465170 

455
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.387622 

456
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.385292 

457
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.388733 

458
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.384079 

459
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.384085 

460
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.38241

Test Error: 
 Accuracy: 87.6%, Avg loss: 0.384102 

591
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.381846 

592
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.374582 

593
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.394335 

594
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.375411 

595
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.379745 

596
Test Error: 
 Accuracy: 87.4%, Avg loss: 0.371262 

597
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.378038 

598
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.383849 

599
Test Error: 
 Accuracy: 87.4%, Avg loss: 0.378885 

600
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.376162 

601
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.377381 

602
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.371676 

603
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.371165 

604
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.377236 

605
Test Error: 
 Accuracy: 87.4%, Avg loss: 0.371314 

606
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.382008 

607
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.37246

Test Error: 
 Accuracy: 87.5%, Avg loss: 0.364519 

738
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.362313 

739
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.363695 

740
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.367506 

741
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.363037 

742
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.375952 

743
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.363868 

744
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.365810 

745
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.360602 

746
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.359989 

747
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.367731 

748
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.375764 

749
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.362573 

750
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.366596 

751
Test Error: 
 Accuracy: 87.4%, Avg loss: 0.366656 

752
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.370732 

753
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.365893 

754
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.37037

Test Error: 
 Accuracy: 88.0%, Avg loss: 0.354575 

885
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.362068 

886
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.353539 

887
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.352527 

888
Test Error: 
 Accuracy: 88.1%, Avg loss: 0.360360 

889
Test Error: 
 Accuracy: 88.1%, Avg loss: 0.355684 

890
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.353183 

891
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.358618 

892
Test Error: 
 Accuracy: 88.3%, Avg loss: 0.354634 

893
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.365358 

894
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.367996 

895
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.352764 

896
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.362088 

897
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.365123 

898
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.357316 

899
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.359481 

900
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.363677 

901
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.35090